# Extract Trajs
Extract trajectories from `.pkl` files after `bad_processing` of `carla_analysis`

In [ ]:
%load_ext autoreload
%autoreload 2

import glob
import numpy as np
import scipy.io as sio
import pickle
import matplotlib.pyplot as plt
import pdb
from datetime import datetime
from tqdm.auto import tqdm

import traceback

from pkl_reader import extract_goals, extract_full_trajectory, interpolate_ego_trajectory

In [ ]:
''' CONFIG: CONSTANTS FOR EXECUTION '''
prune_start=True          # remove stationary portion of ego's trajectory at the start
prune_end=True            # remove stationary portion of ego's trajectory at the end
min_vel_thresh=0.01       # velocity threshold (m/s) above which ego is considered moving
exclude_collisions=True  # return an empty trajectory if there was a collision

dt = 0.1

In [ ]:
save_ext = 'pkl'
file_prefix = '../../carla/PythonAPI/examples/bags/'
search_str = file_prefix + '*.' + save_ext 
files_to_process = glob.glob(search_str)
print('Found %d files to read: %s' % (len(files_to_process), files_to_process))

In [ ]:
''' Extract trajectories'''
num_traj = 0

parking_lot = None
ego_dims    = None

all_goals              = []
all_parking_lot        = []
all_ego_dims           = []
all_ego_trajectory     = []
all_static_object_list = []

for file in tqdm(files_to_process):
    if save_ext == 'pkl':
        res_dict = pickle.load(open(file,'rb'))
    else:
        raise NotImplemented('Invalid Extension')
    
    '''List of [x, y, 0/1] => center position and occupancy (1 free/ 0 occupied)'''
    goals = extract_goals(res_dict)
    
    '''All parking lines as rectangles, [x_center,y_center,dx,dy,th]'''
    parking_lot = res_dict['parking_lot']
    
    '''Dict {'length', 'width'}'''
    ego_dims = res_dict['ego_dimensions']
    
    try:
        assert goals.shape[0] == 32, "Invalid goal shape."
        assert len(res_dict['vehicle_object_lists'][0]) == 56, "Wrong number of static vehicles."
        
        # parse one demonstration
        ego_trajectory, start_ind, switch_ind, end_ind, goal_ind = \
             extract_full_trajectory(res_dict, goals, prune_start, prune_end, \
                                     min_vel_thresh, exclude_collisions)
        
        start_time = ego_trajectory[start_ind, 0]
        end_time = ego_trajectory[end_ind, 0]
        
        '''New time stamps, dt = 0.1'''
        t_interp = np.arange(start_time, end_time, dt)
        
        '''(t_interp-t_interp[0], x_interp, y_interp, heading_interp, v_interp, yawrate_interp)'''
        interp_ego_traj = interpolate_ego_trajectory(ego_trajectory, t_interp, switch_ind, include_intent = False)
        
        '''All static vehicles as rectangles [x_center,y_center,dx,dy,th]'''
        static_object_list = res_dict['static_object_list']
        
        num_traj += 1
        
        all_goals.append(goals)
        all_parking_lot.append(parking_lot)
        all_ego_dims.append(ego_dims)
        all_ego_trajectory.append(interp_ego_traj)
        all_static_object_list.append(static_object_list)
        
    except Exception as e:
        print(file, e)
        traceback.print_exc()
        
print("Number of trajectories out of all data bags: %d / %d" % (num_traj, len(files_to_process)) )

In [ ]:
'''Save to MAT file'''
matfile = 'traj_data.mat'
sio.savemat(matfile, {'goals': all_goals, 
                      'parking_lot': all_parking_lot, 
                      'ego_dims': all_ego_dims, 
                      'ego_trajectory': all_ego_trajectory, 
                      'static_object_list': all_static_object_list})

## Plot and test

In [ ]:
ego_trajectory = all_ego_trajectory[0]
plt.figure()
plt.plot(ego_trajectory[:,1], ego_trajectory[:,2], linewidth=2)